Descartes doesn't have Landsat Surface Reflectance in Collection form, so we are using LSRU: https://github.com/ecohydro/lsru

In [ ]:
import shapely as shp
import geojson
import geopandas as gpd
import os
from lsru import Usgs
from lsru import Espa
from pprint import pprint
import datetime
import time
import yaml

Download NHD water boundary from https://viewer.nationalmap.gov/basic/#productSearch

and convert shapefiles to geojson with ogr2ogr (see bash-scripts folder). Then we can get the bounding box and possibly use the geojson later for other programmatic tools.

In [ ]:
WBD8 = gpd.read_file(wbd_national_path, driver='FileGDB', layer='WBDHU8')

In [ ]:
bbox = list(WBD8[WBD8['NAME']=="Upper North Platte"].envelope.boundary.bounds.iloc[0])

In [ ]:
usgs.get_collection_name(8)

In [ ]:
help(usgs)

In [ ]:
# Instantiate Usgs class and login. requires setting config with credentials
usgs = Usgs()
usgs.login()

In [ ]:
# Instantiate Usgs class and login. requires setting config with credentials
usgs = Usgs()
usgs.login()

# Query the Usgs api to find scene intersecting with the spatio-temporal window
# help(usgs.search)
scene_list = usgs.search(collection='LANDSAT_TM_C1',
                         bbox=bbox,
                         begin=datetime.datetime(2005,1,1),
                         end=datetime.datetime(2006,1,1),
                         max_results=20,
                         max_cloud_cover=10)
# Extract Landsat scene ids for each hit from the metadata
scene_list = [x['displayId'] for x in scene_list]


In [ ]:
filtered = scene_list

Currently, best workflow is to check which path/rows intersect your boundary manually with https://landsat.usgs.gov/pathrow-shapefiles

In [ ]:
## pathrow_list_scsp = ['037037','036037','035037', '035038', '036038'] #path rows for santa clara-sanpedro HUC-6 NHD WBD
pathrow_list_western_nb = ['032031','033031']
kept_scenes = []
filtered = [scene for scene in scene_list if any(good_pathrow in scene for good_pathrow in pathrow_list_western_nb)]
print(len(filtered))
print(len(scene_list))

In [ ]:
bad_dates = ['20160223']
kept_scenes = []
filtered = [scene for scene in filtered if not any(bad_date in scene for bad_date in bad_dates)]

In [ ]:
filtered

In [ ]:
espa.get_available_products(filtered)

In [ ]:
# Instantiate Espa class
espa = Espa()
espa.get_available_products(filtered)
# Place order (full scenes, no reprojection, sr and pixel_qa)
order = espa.order(scene_list=filtered, products=['sr'])
print(order.orderid)

In [ ]:
help(espa)

In [ ]:
espa = Espa()
help(espa.orders[0])

Only use this to check relatively small orders that are in the 10s of scenes. Orders in the 100s of scenes will take longer, particularly for Landsat 8 surface reflectance.

In [ ]:
for order in espa.orders:
    while order.is_complete==False:
    # Orders have their own class with attributes and methods
        print('%s: %s' % (order.orderid, order.status))
        time.sleep(120)

In [ ]:
espa = Espa()


In [ ]:
espa.orders

In [ ]:
help(espa.orders[0].download_all_complete)

In [ ]:
os.getcwd()

In [ ]:
for order in espa.orders:
    if order.is_complete:
        order.download_all_complete("./data/landsat-8-sr/", unpack=True)

In [ ]:
import rasterio as rast
import rasterio.plot as rsplot
import geopandas as gpd
src=rast.open(os.path.join(LANDSAT_DIR,"LC08_L1TP_037037_20140112_20170307_01_T1_sr_band3.tif"))
fig,ax=plt.subplots()
rsplot.show(src,ax=ax)

plt.show()

Below are some descartes labs methods for querying info and scenes, but Descartes doesn't have collection level 2 surface reflectance (only precollection, before April 2017)

Use Descartes to view Landsat Scenes that intersect with the bbox.

In [ ]:
from descarteslabs.client.services import Metadata
products = Metadata().available_products()
products

In [ ]:
clean_aoi_geometry = {}

In [ ]:
coordinates = eval(watershed_bbox.to_json())['features'][0]['geometry']['coordinates'][0]
for i,c in enumerate(coordinates):
    coordinates[i]=tuple(c)
coordinates = tuple(coordinates)
clean_aoi_geometry['coordinates'] = (coordinates,)
clean_aoi_geometry['type']='Polygon'

In [ ]:
clean_aoi_geometry

In [ ]:
clean_aoi_geometry['coordinates']=eval(watershed_bbox.to_json())['features'][0]['geometry']['coordinates'][0]

In [ ]:
clean_aoi_geometry['type']='Polygon'

In [ ]:
clean_aoi_geometry

In [ ]:

scenes, ctx = dl.scenes.search(clean_aoi_geometry,
                              products=['landsat:LC08:PRE:TOAR'],
                              start_datetime="2000-05-01",
                              end_datetime="2018-08-01",
                              limit=1000)

In [ ]:
scenes.sorted("properties.date")

In [ ]:
for i in scenes.each.properties.id:
    print(i)